In [1]:
import torch
import random

from util import TokenizerUtil

tokenizer = TokenizerUtil()

input_ids, attention_mask = tokenizer.encode('how are you', max_length=4)

input_ids, attention_mask, tokenizer.decode(input_ids)

/root/miniconda3/envs/cuda117/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(tensor([   0, 9178,   32,    2]), tensor([1, 1, 1, 1]), '<s>how are</s>')

In [2]:
from datasets import load_dataset
from transformers import default_data_collator

dataset = load_dataset('json', data_files='dataset/train.json', split='train')

#2,4,4切分,取第0部分
dataset = dataset.select(range(15000))


def f(data):
    #随机生成两种回答
    if random.random() > 0.5:
        data['chosen'] = data['chosen'].swapcase()
    data = data['prompt'] + data['chosen']

    input_ids, attention_mask = tokenizer.encode(data)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': input_ids.clone()
    }


dataset = dataset.map(f, remove_columns=dataset.column_names)

loader = torch.utils.data.DataLoader(dataset,
                                     collate_fn=default_data_collator,
                                     batch_size=4,
                                     shuffle=True,
                                     drop_last=True)

len(loader), next(iter(loader))

(3750,
 {'input_ids': tensor([[    0, 33837,    35,  ...,     1,     1,     1],
          [    0, 33837,    35,  ...,     1,     1,     1],
          [    0, 33837,    35,  ...,     1,     1,     1],
          [    0, 33837,    35,  ...,     1,     1,     1]]),
  'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0]]),
  'labels': tensor([[    0, 33837,    35,  ...,     1,     1,     1],
          [    0, 33837,    35,  ...,     1,     1,     1],
          [    0, 33837,    35,  ...,     1,     1,     1],
          [    0, 33837,    35,  ...,     1,     1,     1]])})

In [3]:
from transformers import AutoModelForCausalLM
from accelerate import Accelerator

model_actor = AutoModelForCausalLM.from_pretrained('model/facebook/opt-350m',
                                                   dropout=0.0)
model_actor.train()

optimizer = torch.optim.Adam(model_actor.parameters(), lr=1e-4)

accelerator = Accelerator(mixed_precision='fp16')

loader, model_actor, optimizer = accelerator.prepare(loader, model_actor,
                                                     optimizer)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [4]:
for i, data in enumerate(loader):
    out = model_actor(**data)
    accelerator.backward(out.loss)
    accelerator.clip_grad_norm_(model_actor.parameters(), 1.0)
    optimizer.step()
    optimizer.zero_grad()

    if (i + 1) % 100 == 0:
        print(i, len(loader), out.loss.item())

        logits = out.logits[0].argmax(1)
        print(tokenizer.decode(logits))

    if i == 1000:
        break

model_actor.save_pretrained('model/actor')

99 3750 0.2825024127960205
Human: context= CREATE TABLE table_name_97 (position VARCHAR, nation VARCHAR) nation VARCHAR) question= What nation have gold than 1 gold?? less silver medals? Assistant:select MATION from TABLE_NAME_87 where GOLD = 1 and 4VER = "</s>
199 3750 0.3939495384693146
Human: context= CREATE TABLE table_name887_1 (result_in_the VEGER, date_title_air VARCHAR, question= What is number is the episode has aired on on 4, 1990? Assistant:select no(no_in_series) FROM table_228973_5 WHERE original_air_date = "April 27, 1999"</s>
299 3750 0.6533144116401672
Human: context= CREATE TABLE table_name_43 (yearzyme VEGER, rank Vclassname VARCHAR) question= What least of by playerer? a is of the in Assistant:select SUM(enrollment) FROM table_name_5 WHERE teamnicknickname = "wildcats"</s>
399 3750 0.4442545771598816
Human: context= CREATE TABLE table_name_51 (scores__ V INT V_classARCHAR, i VARCHAR) question= What isHSAA class class is you gameals with at the Assistant:select IHSAA_